In [221]:
import pandas as pd
import numpy as np
import json
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
import random

print("imports happened")

imports happened


**This notebook is going to cluster neighborhoods in Toronto based on venues they have in common. Huh. Taking code almost verbatim from Manhattan clustering lab.**

**Read back in file with Toronto neighborhoods and lat/long**

In [222]:
df = pd.read_csv('toronto_with_latlng.csv')
df.drop('Unnamed: 0', axis = 1, inplace = True)
df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


**How many unique borough postal codes are there and what are they?**

In [223]:
df2 = df.groupby('Borough').count()
df2

,PostalCode,Neighbourhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,18,18,18,18
East Toronto,5,5,5,5
East York,5,5,5,5
Etobicoke,12,12,12,12
Mississauga,1,1,1,1
North York,24,24,24,24
Queen's Park,1,1,1,1
Scarborough,17,17,17,17


**I've decided to do downtown Toronto only to reduce the number of Foursquare calls I need to make.**


In [224]:
df_downtown = df[df['Borough'] == 'Downtown Toronto'].reset_index(drop = True)
df_downtown.drop(['PostalCode'], inplace = True, axis = 1)
df_downtown.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Downtown Toronto,Rosedale,43.679563,-79.377529
1,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
4,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [225]:
df_downtown.sort_values(by = ['Neighbourhood'], inplace = True)

df_downtown.head()

,Borough,Neighbourhood,Latitude,Longitude
8,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
6,Downtown Toronto,Berczy Park,43.644771,-79.373306
14,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420
1,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
7,Downtown Toronto,Central Bay Street,43.657952,-79.387383


**Just checking the size**

In [226]:
df_downtown.shape

(18, 4)

**Now I suppose I have to do a map. ll for downtown toronto taken from google search.**

In [227]:
downtown_latitude = 43.6532
downtown_longitude = -79.3832

downtown_map = folium.Map(location=[downtown_latitude, downtown_longitude], zoom_start=14)
for lat, lng, borough, neighbourhood in zip(df_downtown['Latitude'], df_downtown['Longitude'], df_downtown['Borough'], df_downtown['Neighbourhood']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(downtown_map)  
downtown_map

**Next, I'm going to get venues from the 'downtown toronto' neighborhoods and start manipulating the info.**

In [228]:
#This function taken verbatim from the Manhattan notebook

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude',
                  'Neighbourhood Longitude',
                  'Venue',           
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

**Run the above function on all nieghborhoods in 'Downtown Toronto'**

In [229]:
CLIENT_ID = '0KE5TTWD03ZGR5NHFKGI2HKDDLIZBQUZB1VUZEBEQIJSIE2N'
CLIENT_SECRET = '41K1PDWHLZXKDV5YB10JA4MICC1LQYCGU5UHIJ0GFEAXJQEU'
VERSION = '20190731'
LIMIT = 100
radius = 500

toronto_venues = getNearbyVenues(names=df_downtown['Neighbourhood'],
    latitudes=df_downtown['Latitude'],longitudes=df_downtown['Longitude'])

Adelaide, King, Richmond
Berczy Park
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Cabbagetown, St. James Town
Central Bay Street
Chinatown, Grange Park, Kensington Market
Christie
Church and Wellesley
Commerce Court, Victoria Hotel
Design Exchange, Toronto Dominion Centre
First Canadian Place, Underground city
Harbord, University of Toronto
Harbourfront East, Toronto Islands, Union Station
Harbourfront, Regent Park
Rosedale
Ryerson, Garden District
St. James Town
Stn A PO Boxes 25 The Esplanade


**See how many venues were returned in total.**

In [230]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16
"Cabbagetown, St. James Town",43,43,43,43,43,43
Central Bay Street,82,82,82,82,82,82
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16
Church and Wellesley,85,85,85,85,85,85
"Commerce Court, Victoria Hotel",100,100,100,100,100,100


**Now, do the slice and dice (plagiarized from the Manhattan notebook).**

In [231]:
unique_venues = len(toronto_venues['Venue Category'].unique())
print('There are {} unique categories.'.format(unique_venues))

There are 203 unique categories.


**The next thing to do is one-hot encoding**

In [232]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood']
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.columns)
toronto_onehot.head()

Index(['Neighbourhood', 'Afghan Restaurant', 'Airport', 'Airport Food Court',
       'Airport Gate', 'Airport Lounge', 'Airport Service', 'Airport Terminal',
       'American Restaurant', 'Antique Shop',
       ...
       'Thrift / Vintage Store', 'Toy / Game Store', 'Trail', 'Train Station',
       'Vegetarian / Vegan Restaurant', 'Video Game Store',
       'Vietnamese Restaurant', 'Wine Bar', 'Wings Joint', 'Yoga Studio'],
      dtype='object', length=204)


,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Adelaide, King, Richmond",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Adelaide, King, Richmond",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Adelaide, King, Richmond",0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,"Adelaide, King, Richmond",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Do some more slicing and dicing**

In [233]:
toronto_onehot.shape

(1281, 204)

In [234]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.030000,0.000000,...,0.00,0.00,0.0,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000
1,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00,0.00,0.0,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.000000,...,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Cabbagetown, St. James Town",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.023256
4,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.012195,0.000000,...,0.00,0.00,0.0,0.00,0.012195,0.000000,0.000000,0.012195,0.000000,0.012195
5,"Chinatown, Grange Park, Kensington Market",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.01,0.00,0.0,0.00,0.060000,0.000000,0.040000,0.010000,0.000000,0.000000
6,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Church and Wellesley,0.011765,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.011765,0.000000,...,0.00,0.00,0.0,0.00,0.000000,0.011765,0.011765,0.000000,0.011765,0.011765
8,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.040000,0.000000,...,0.00,0.00,0.0,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
9,"Design Exchange, Toronto Dominion Centre",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.040000,0.000000,...,0.00,0.00,0.0,0.01,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000


In [235]:
toronto_grouped.shape

(18, 204)

**print each neighborhood along with the top 5 most common venues**

In [236]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2       Steakhouse  0.04
3              Bar  0.04
4  Thai Restaurant  0.04


----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1    Cocktail Bar  0.05
2          Bakery  0.05
3  Farmers Market  0.04
4            Café  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airport Service  0.19
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3          Boutique  0.06
4   Harbor / Marina  0.06


----Cabbagetown, St. James Town----
                venue  freq
0         Coffee Shop  0.07
1                 Pub  0.05
2  Italian Restaurant  0.05
3                Café  0.05
4          Restaurant  0.05


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.15
1  Italian Restaurant  0.05
2      Ice Cream Shop  0.05
3                Café  0.04
4  

**Create a pandas dataframe of the top ten venues for each neighbourhood after sorting according to count**

In [237]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [238]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,Breakfast Spot,Hotel,Burger Joint,Gym,Asian Restaurant
1,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Farmers Market,Cheese Shop,Beer Bar,Seafood Restaurant,Steakhouse,Café,Diner
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Bar,Coffee Shop,Sculpture Garden,Airport Food Court
3,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Pub,Café,Italian Restaurant,Pizza Place,Bakery,Outdoor Sculpture,Butcher,Caribbean Restaurant
4,Central Bay Street,Coffee Shop,Ice Cream Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Bubble Tea Shop,Sushi Restaurant,Salad Place,Indian Restaurant


**Merge the df with venues to the df with lat/long**

In [239]:
df_downtown.head()

,Borough,Neighbourhood,Latitude,Longitude
8,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
6,Downtown Toronto,Berczy Park,43.644771,-79.373306
14,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420
1,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
7,Downtown Toronto,Central Bay Street,43.657952,-79.387383


**Now it's time to cluster. Form 5 clusters using k-means**

In [240]:
toronto_joined = toronto_latlng.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on = 'Neighbourhood')
toronto_joined.head()

,PostalCode,Borough,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Neighbourhood,,,,,,,,,,,,,,
"Adelaide, King, Richmond",M5H,Downtown Toronto,43.650571,-79.384568,Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,Breakfast Spot,Hotel,Burger Joint,Gym,Asian Restaurant
Berczy Park,M5E,Downtown Toronto,43.644771,-79.373306,Coffee Shop,Bakery,Cocktail Bar,Farmers Market,Cheese Shop,Beer Bar,Seafood Restaurant,Steakhouse,Café,Diner
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",M5V,Downtown Toronto,43.628947,-79.394420,Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Bar,Coffee Shop,Sculpture Garden,Airport Food Court
"Cabbagetown, St. James Town",M4X,Downtown Toronto,43.667967,-79.367675,Coffee Shop,Restaurant,Pub,Café,Italian Restaurant,Pizza Place,Bakery,Outdoor Sculpture,Butcher,Caribbean Restaurant
Central Bay Street,M5G,Downtown Toronto,43.657952,-79.387383,Coffee Shop,Ice Cream Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Bubble Tea Shop,Sushi Restaurant,Salad Place,Indian Restaurant


**check that toronto_group is sorted so label come out in right order**

In [241]:
toronto_grouped.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.030000,0.0,...,0.0,0.0,0.0,0.0,0.010000,0.0,0.0,0.010000,0.0,0.000000
1,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.017544,0.0,0.0,0.000000,0.0,0.000000
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
3,"Cabbagetown, St. James Town",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.023256
4,Central Bay Street,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.012195,0.0,...,0.0,0.0,0.0,0.0,0.012195,0.0,0.0,0.012195,0.0,0.012195


In [242]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 3, 0, 0, 2, 4, 0, 0, 0], dtype=int32)

In [243]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,Breakfast Spot,Hotel,Burger Joint,Gym,Asian Restaurant
1,0,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Farmers Market,Cheese Shop,Beer Bar,Seafood Restaurant,Steakhouse,Café,Diner
2,3,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Bar,Coffee Shop,Sculpture Garden,Airport Food Court
3,0,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Pub,Café,Italian Restaurant,Pizza Place,Bakery,Outdoor Sculpture,Butcher,Caribbean Restaurant
4,0,Central Bay Street,Coffee Shop,Ice Cream Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Bubble Tea Shop,Sushi Restaurant,Salad Place,Indian Restaurant


**Create a map of the clusters**

In [244]:
map_clusters = folium.Map(location=[downtown_latitude, downtown_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_downtown['Latitude'], df_downtown['Longitude'], neighborhoods_venues_sorted['Neighbourhood'], neighborhoods_venues_sorted['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Create a new dataframe out of the top 20 for each neighborhood and their cluster labels**

In [245]:
df_new = neighborhoods_venues_sorted.join(df_downtown.set_index('Neighbourhood'), on = 'Neighbourhood')
df_new.head()

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Borough,Latitude,Longitude
0,0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,Breakfast Spot,Hotel,Burger Joint,Gym,Asian Restaurant,Downtown Toronto,43.650571,-79.384568
1,0,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Farmers Market,Cheese Shop,Beer Bar,Seafood Restaurant,Steakhouse,Café,Diner,Downtown Toronto,43.644771,-79.373306
2,3,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Bar,Coffee Shop,Sculpture Garden,Airport Food Court,Downtown Toronto,43.628947,-79.394420
3,0,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Pub,Café,Italian Restaurant,Pizza Place,Bakery,Outdoor Sculpture,Butcher,Caribbean Restaurant,Downtown Toronto,43.667967,-79.367675
4,0,Central Bay Street,Coffee Shop,Ice Cream Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Bubble Tea Shop,Sushi Restaurant,Salad Place,Indian Restaurant,Downtown Toronto,43.657952,-79.387383


**different dfs for each cluster**

**How many in each cluster?**

In [246]:
df_new['Cluster Labels'].value_counts()


0    13
2     2
4     1
3     1
1     1
Name: Cluster Labels, dtype: int64

In [247]:
df_new.head()

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Borough,Latitude,Longitude
0,0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,Breakfast Spot,Hotel,Burger Joint,Gym,Asian Restaurant,Downtown Toronto,43.650571,-79.384568
1,0,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Farmers Market,Cheese Shop,Beer Bar,Seafood Restaurant,Steakhouse,Café,Diner,Downtown Toronto,43.644771,-79.373306
2,3,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Bar,Coffee Shop,Sculpture Garden,Airport Food Court,Downtown Toronto,43.628947,-79.394420
3,0,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Pub,Café,Italian Restaurant,Pizza Place,Bakery,Outdoor Sculpture,Butcher,Caribbean Restaurant,Downtown Toronto,43.667967,-79.367675
4,0,Central Bay Street,Coffee Shop,Ice Cream Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Bubble Tea Shop,Sushi Restaurant,Salad Place,Indian Restaurant,Downtown Toronto,43.657952,-79.387383


In [248]:
df_new.loc[df_new['Cluster Labels'] == 0, df_new.columns[[1] + list(range(2, df_new.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Borough,Latitude,Longitude
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,Breakfast Spot,Hotel,Burger Joint,Gym,Asian Restaurant,Downtown Toronto,43.650571,-79.384568
1,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Farmers Market,Cheese Shop,Beer Bar,Seafood Restaurant,Steakhouse,Café,Diner,Downtown Toronto,43.644771,-79.373306
3,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Pub,Café,Italian Restaurant,Pizza Place,Bakery,Outdoor Sculpture,Butcher,Caribbean Restaurant,Downtown Toronto,43.667967,-79.367675
4,Central Bay Street,Coffee Shop,Ice Cream Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Bubble Tea Shop,Sushi Restaurant,Salad Place,Indian Restaurant,Downtown Toronto,43.657952,-79.387383
7,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Gastropub,Pizza Place,Men's Store,Mediterranean Restaurant,Hotel,Downtown Toronto,43.665860,-79.383160
8,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gastropub,Bakery,Steakhouse,Seafood Restaurant,Gym,Downtown Toronto,43.648198,-79.379817
9,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Café,Hotel,Italian Restaurant,Restaurant,American Restaurant,Bar,Gym,Deli / Bodega,Gastropub,Downtown Toronto,43.647177,-79.381576
10,"First Canadian Place, Underground city",Coffee Shop,Café,Steakhouse,Hotel,Restaurant,Bar,Seafood Restaurant,Gym,American Restaurant,Deli / Bodega,Downtown Toronto,43.648429,-79.382280
12,"Harbourfront East, Toronto Islands, Union Station",Coffee Shop,Hotel,Aquarium,Café,Italian Restaurant,Scenic Lookout,Pizza Place,Bakery,Brewery,Sporting Goods Shop,Downtown Toronto,43.640816,-79.381752
13,"Harbourfront, Regent Park",Coffee Shop,Bakery,Café,Pub,Park,Breakfast Spot,Restaurant,Mexican Restaurant,Historic Site,Cosmetics Shop,Downtown Toronto,43.654260,-79.360636


In [249]:
df_new.loc[df_new['Cluster Labels'] == 1, df_new.columns[[1] + list(range(2, df_new.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Borough,Latitude,Longitude
14,Rosedale,Park,Playground,Trail,Building,Yoga Studio,Diner,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Downtown Toronto,43.679563,-79.377529


In [250]:
df_new.loc[df_new['Cluster Labels'] == 2, df_new.columns[[1] + list(range(2, df_new.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Borough,Latitude,Longitude
5,"Chinatown, Grange Park, Kensington Market",Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Bar,Mexican Restaurant,Dumpling Restaurant,Bakery,Vietnamese Restaurant,Coffee Shop,Park,Downtown Toronto,43.653206,-79.400049
11,"Harbord, University of Toronto",Café,Restaurant,Bar,Japanese Restaurant,Bookstore,Bakery,Gym,Beer Bar,Beer Store,Nightclub,Downtown Toronto,43.662696,-79.400049


In [251]:
df_new.loc[df_new['Cluster Labels'] == 3, df_new.columns[[1] + list(range(2, df_new.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Borough,Latitude,Longitude
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Bar,Coffee Shop,Sculpture Garden,Airport Food Court,Downtown Toronto,43.628947,-79.39442


In [252]:
df_new.loc[df_new['Cluster Labels'] == 4, df_new.columns[[1] + list(range(2, df_new.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Borough,Latitude,Longitude
6,Christie,Café,Grocery Store,Park,Nightclub,Diner,Restaurant,Baby Store,Italian Restaurant,Athletics & Sports,Coffee Shop,Downtown Toronto,43.669542,-79.422564


Cluster 0 appears to be the 'real' downtown with lots of entries.
Cluster 1 could be residential for immigrants (ethnic restaurants)
Cluster 2 seems to also have a varied set of venues 

I'm not going to do any more because I'm tired of this and I don't really like this type of analysis because I don't think it says anything.

In [253]:
print("done")

done
